In [165]:
## BAZAR POUR INITIALISER

from IPython.core.display import display, HTML
import json
import numpy as np

def plot3D(X, Y, Z, height=600, xlabel = "X", ylabel = "Y", zlabel = "Z", initialCamera = None):

    options = {
        "width": "100%",
        "style": "surface",
        "showPerspective": True,
        "showGrid": True,
        "showShadow": False,
        "keepAspectRatio": True,
        "height": str(height) + "px"
    }

    if initialCamera:
        options["cameraPosition"] = initialCamera
        
    data = [ {"x": X[y,x], "y": Y[y,x], "z": Z[y,x]} for y in range(X.shape[0]) for x in range(X.shape[1]) ]
    visCode = r"""
       <link href="https://cdnjs.cloudflare.com/ajax/libs/vis/4.21.0/vis.min.css" type="text/css" rel="stylesheet" />
       <script src="https://cdnjs.cloudflare.com/ajax/libs/vis/4.21.0/vis.min.js"></script>
       <div id="pos" style="top:0px;left:0px;position:absolute;"></div>
       <div id="visualization"></div>
       <script type="text/javascript">
        var data = new vis.DataSet();
        data.add(""" + json.dumps(data) + """);
        var options = """ + json.dumps(options) + """;
        var container = document.getElementById("visualization");
        var graph3d = new vis.Graph3d(container, data, options);
        graph3d.on("cameraPositionChange", function(evt)
        {
            elem = document.getElementById("pos");
            elem.innerHTML = "H: " + evt.horizontal + "<br>V: " + evt.vertical + "<br>D: " + evt.distance;
        });
       </script>
    """
    htmlCode = "<iframe srcdoc='"+visCode+"' width='100%' height='" + str(height) + "px' style='border:0;' scrolling='no'> </iframe>"
    display(HTML(htmlCode))

# Exemple 1 :  $f(x,y)=1- x^2-y^2$

In [166]:
def f(x,y):
    u1 = 1-x**2-y**2
    u2 = (x**2+y**2<=1)
    return np.multiply(u1,u2)
X1= np.linspace(-1, 1,50)
Y1 = np.linspace(-1, 1,50)
X1, Y1 = np.meshgrid(X1, Y1)
Z1 = f(X1,Y1)

In [167]:
plot3D(X1,Y1,Z1) # z = 1 - x**2 - y**2

# Exemple 2 : $f(x,y)=x^2-y^2$

In [168]:
X2= np.linspace(-2, 2, 50)
Y2 = np.linspace(-2, 2, 50)
X2, Y2 = np.meshgrid(X2, Y2)
Z2 = X2**2-Y2**2

In [169]:
plot3D(X2,Y2,Z2) # z = x**2 - y**2

# Exemple 3 : $f(x,y)= \sin\left( 2\pi\left( \dfrac{x}{2}+\dfrac{y}{4}\right)\right)$

In [171]:
X3= np.linspace(-2, 2, 50)
Y3 = np.linspace(-2, 2, 50)
X3, Y3 = np.meshgrid(X3, Y3)
Z3 = np.sin(2*np.pi*(0.5*X3+0.25*Y3))

In [172]:
plot3D(X3,Y3,Z3) # z = sin(2*pi(0.5*x + 0.25*y))

# Exemple 4 : advection $\dfrac{\partial u}{\partial t}+v\dfrac{\partial u}{\partial x}=0$

## Tracé de la surface (statique)

In [174]:
def u0(t,x):
    v = 1 
    return 1/(1+(x-v*t)**2)
T2 = np.linspace(0,5,50)      
X2 = np.linspace(-10,10,50)    
u0 = np.vectorize(u0) 
T2,X2 = np.meshgrid(T2,X2)       
Z2 = u0(T2,X2)   

In [175]:
plot3D(T2,X2,Z2) # Tracé de u(t,x) = (x-v*t)**(-2)

$u(t,x)= \dfrac{1}{(x-vt)^2}$

## Point de vue évolution (dynamique)

In [176]:
# 1. définition du cadre
fig, ax = plt.subplots()
ax.set_xlim((-5, 5))
ax.set_ylim((0,2.5))
ax.grid('on')
line, = ax.plot([], [], lw=2)
# 2. Animation (appels successifs)
def animate(i):
    x = np.linspace(-10,10, 1000)
    y =1/(1+(x-0.01*i)**2)
    ax.set_title(r'$u(t,x) = \frac{{1}}{{(x-vt^2)}}$ t={}'.format(i/100))
    line.set_data(x, y)
    return (line,)
#3. Fonction d'initialisation (dessine le cadre pour chaque image)
def init():
    line.set_data([], [])
    return (line,)
#4.  Appel de l'animateur. blit=True pour ne redessiner que ce qui a changé
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=500, interval=50, blit=True)

In [177]:
HTML(anim.to_html5_video())

# Exemple 5 : diffusion $\dfrac{\partial u}{\partial t}-D\dfrac{\partial u}{\partial x^2}=0$

## Point de vue statique

In [140]:
def f0(x,t):
    pi =np.pi
    Q = 5
    D = 1/(4*pi)
    return (5/(np.sqrt(4*pi*D*t)))*np.exp(-x**2/(4*D*t))
T2 = np.linspace(0.1,10,50)      
X2 = np.linspace(-7,7,50)    
f0 = np.vectorize(f0)
X2,T2 = np.meshgrid(X2,T2)       
Z2 = f0(X2,T2)   

In [149]:
plot3D(T2,X2,Z2) # Tracé du noyau de la chaleur

## Point de vue dynamique

In [152]:
# 1. définition du cadre
fig, ax = plt.subplots()
ax.set_xlim((-7, 7))
ax.set_ylim((0,8))
ax.grid('on')
line, = ax.plot([], [], lw=2)
# 2. Animation (appels successifs)
def animate(i):
    pi =np.pi
    Q = 5
    D = 1/(4*pi)
    x = np.linspace(-7,7, 1000)
    y =(5/(np.sqrt(4*pi*D*(0.01*(i+1))))*np.exp(-x**2/(4*D*(0.01*(i+1)))))
    ax.set_title(r'$u(t,x) =\frac{{Q}}{{\sqrt{{4\pi D t}}}}\
                    exp\left(\frac{{-x^2}}{{4Dt}}\right)$ t={}'.format(i/100))
    line.set_data(x, y)
    return (line,)
#3. Fonction d'initialisation (dessine le cadre pour chaque image)
def init():
    line.set_data([], [])
    return (line,)
#4.  Appel de l'animateur. blit=True pour ne redessiner que ce qui a changé
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=500, interval=50, blit=True)

In [153]:
HTML(anim.to_html5_video())

$u(t,x) =\dfrac{Q}{\sqrt{4\pi D t}}\exp\left(\dfrac{-x^2}{4Dt}\right)$